In [1]:
# import os 

# os.environ['LD_LIBRARY_PATH'] = '/media/Data/Home/PanStarrs/Jan/HI/augment/SBF/codes/pysbf:' + \
#                                 os.environ["LD_LIBRARY_PATH"]

# os.environ['LD_LIBRARY_PATH'] 

In [5]:
import sys
sys.path.insert(0, "../")
import time

from pysbf import *

from IPython.display import display, Markdown, clear_output
warnings.filterwarnings('ignore')

In [2]:
## This is the folder that holds recent observations by wfc3 on HST in a SNAP program

inFolder = '/media/Data/Home/PanStarrs/Jan/HI/augment/SBF/wfc3-16262/'
configFolder = '/media/Data/Home/PanStarrs/Jan/HI/augment/SBF/codes/notebooks/config/'

name = "u03396"

In [3]:
inFolder = inFolder + '/'
configFolder   = configFolder + '/'

outFolder = "Outputs_"+name+'/'
logFile = outFolder+name+"_model_log.csv"


if exists(logFile):
    with open(logFile, 'r') as f:
        line = f.readline()

    if line.strip("#").strip()[:4] == "uuid":
        old_uuid = line.strip("#").split(":")[1].strip()                   
        if os.path.isdir(outFolder+name+"_"+old_uuid):
            uuid = old_uuid
            params = get_obj_params(open_log_df(logFile))

else:
    print("There is no log-file. \nPlease run elliprof to generate the log file and the relevant plots.")
    print("Elliprof notebook name: \"Run_elliprof_function.ipynb\"")

In [4]:
df_log = open_log_df(logFile)
sky_med = float(df_log.loc["initial_sky_med"].value)

df = open_log_df(logFile)
obj_uuid = df.loc['uuid'].value
objRoot = outFolder+name+'_'+obj_uuid+'/'

X0 = float(df.loc['X0'].value)
Y0 = float(df.loc['Y0'].value)

Config = {} 

Config["name"] = name
Config["inFolder"] = inFolder
Config["objRoot"] = objRoot
Config["configFolder"] = configFolder
Config["X0"] = X0
Config["Y0"] = Y0

In [5]:
SKY = df.loc["sky"].value

SKY

'2245'

In [6]:
hdu_list = fits.open(inFolder + "{}/{}j.fits".format(name, name))
image_data = hdu_list[0].data

TOP = str(int(np.ceil(np.max(image_data))))       # 
BOTTOM = str(int(np.floor(np.min(image_data))))   # ~1000, -2000

In [7]:
BOTTOM

'-771'

In [8]:
model = 0   # the main model

suffix = ".%03d" % model

In [9]:
THRESHMAX   = TOP # 1850000   # around saturation level
THRESHMIN   = 1000  
 
SCALEAPRADIUS = 3.0

RDNOISE = 200    
FWHM    = 1.4    # same most of the time

In [10]:
IMAGE_IN    = objRoot+ "resid" + suffix   ## can be resid + (extended objects + Dmask --> dophot)
PARAMS_OUT  = objRoot+ "opar" + suffix    

IMAGE_VAR   = objRoot+ "dvar" + suffix  ## renuc factor 2x the one used for sextractor
IMAGE_OUT   = objRoot+ "dpr" + suffix   ## temp file to use in ds9
OBJECTS_OUT = objRoot+ "dpo" + suffix   ## the most important
FINISHFILE  = objRoot+ "dfin" + suffix  ## remove this (for running dophot mulitple times)

In [11]:
## making dpar files

In [12]:
dophot_params = """
= """ + name + """ GO-16262
SKY = """+SKY+"""     Approximate mean sky value in data numbers.
FWHM = """+str(FWHM)+"""             Approx FWHM of objects (pixels) along major axis.
AXIS_RATIO = 0.9       For star objects.  AR=b/a; b=minor axis.
TILT = 90.0            Angle of major axis in degrees; +x=0; +y=90.
EPERDN = 1.0           Electrons per data number.
RDNOISE = """+str(RDNOISE)+"""         Readout noise in electrons.
TOP = """+TOP+"""          Maximum allowed data value in data numbers.
=
AUTOSCALE = 'YES'      Auto-scaling of fitting and masking radii by FWHM.
AUTOSCALEAP = 'YES'    Auto-scaling of star and sky aperture radii by FWHM.
AUTOTHRESH = 'NO'      Auto-scaling of thresholds and bottom.
AUTOBLIT = 'YES'       Auto-scaling of obliteration parameters.
AUTOMEDSCALE = 'YES'   Auto-scaling for median filter box size.
= If autothresh=no take these parameters.
BOTTOM = """+BOTTOM+"""       Lowest allowed data value in data numbers.
THRESHMAX = """+str(THRESHMAX)+"""    Value of maximum threshold.
THRESHMIN = """+str(THRESHMIN)+"""     Value of lowest threshold.
=
PARAMS_DEFAULT = '"""+configFolder+"""dophot/paramdefault'   Default parameters file name.
PARAMS_OUT = '"""+PARAMS_OUT+"""'           Output parameters file name.
IMAGE_IN = '"""+IMAGE_IN+"""'            Input image name. 
=  If desired a file of extra variance can be specified, which is
=  added to the normal variance of (DATA/eADU)+(RN/eADU)^2.  Note
=  that it must have proper ADU^2 units of variance 
IMAGE_VAR = '"""+IMAGE_VAR+"""'            Input extra variance image
IMAGE_OUT = '"""+IMAGE_OUT+"""'             Output image name.
OBJECTS_OUT = '"""+OBJECTS_OUT+"""'           Output object list file name.
CENTINTMAX = """+TOP+"""              Obliterate if central intensity exceeds this.
TOPSAT = """+TOP+"""                  Ignore pixels with higher counts in saturated stars.
=
OBJTYPE_IN = 'INTERNAL'     Input format: (COMPLETE, INTERNAL, BINARY)
OBJTYPE_OUT = 'COMPLETE'    Output format: (COMPLETE, INTERNAL,INCOMPLETE, BINARY,DAOPHOT)
=   If autoscaleap=yes determine parameters using
SCALEAPRADIUS = """+str(SCALEAPRADIUS)+"""    Size of aperture phot radius in units of FWHM.
END
"""


In [13]:
DPAR_FILE = objRoot+ "dpar" + suffix

with open(DPAR_FILE, 'w') as f:
    f.write(dophot_params)

In [14]:
DPAR_FILE

'Outputs_u03396/u03396_bb6d9894f133/dpar.000'

In [16]:
inFolder

'/media/Data/Home/PanStarrs/Jan/HI/augment/SBF/wfc3-16262//'

In [ ]:
 Using Mv = -7.40 for GCs (NOT for JHK)


In [ ]:
negative distance --> fitting first 

Enter new bright end cutoff (22.00):

sigma width of the GC LF in units of mag ~1.2 small objects ~1.3 ~1.4 giant ellipticals

In [ ]:
LIKENEW6 -- IR version 2020
 Enter object list file name (from mkincat): se_lknj.000
 Enter color (B/V/R/I/z/J/H/K/Ks = 0/1/2/3/4/5/6/7/8) of data: 5
 Enter scale ("/p), psf [-]fwhm(p), and [-]dist (Mpc): 0.128, 1.4, 60
 Enter cutoff [-]S/N ratio and T/F for soft bias correction:-4.5


In [ ]:
ptm6 --> modified manually ptm6b